<a href="https://colab.research.google.com/github/Basifrank/HNG12_Codes/blob/main/Copy_of_STAGE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import geopy
from geopy.geocoders import ArcGIS
geolocator_arcgis = ArcGIS()

import warnings
warnings.filterwarnings('ignore')


In [ ]:
imo_crosscheck = pd.read_csv("/content/drive/MyDrive/HNG INTERNSHIP/HNG 12/Stage 8/IMO_crosschecked.csv")


lagos_crosscheck = pd.read_csv("/content/drive/MyDrive/HNG INTERNSHIP/HNG 12/Stage 8/LAGOS_crosschecked.csv")
#imo_polling_unit = pd.read_csv("/content/drive/MyDrive/HNG INTERNSHIP/HNG 11/STAGE 1/Imo_polling_units.csv")

In [ ]:
lagos_crosscheck.columns

Index(['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Accredited_Voters',
       'Registered_Voters', 'Results_Found', 'Transcription_Count',
       'Result_Sheet_Stamped', 'Result_Sheet_Corrected',
       'Result_Sheet_Invalid', 'Result_Sheet_Unclear', 'Result_Sheet_Unsigned',
       'APC', 'LP', 'PDP', 'NNPP', 'Results_File'],
      dtype='object')

In [ ]:
imo_crosscheck.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File
0,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-001,ALADINMA UMUELEM HALL,175,905,True,-1,False,False,False,False,UNKNOWN,9,40,100,3,https://docs.inecelectionresults.net/elections...
1,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-002,OKWUAKU HALL,243,852,True,-1,False,False,False,False,UNKNOWN,3,230,9,0,https://docs.inecelectionresults.net/elections...
2,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-003,UMUNKWO VILLAGE HALL,183,704,True,-1,False,False,False,False,UNKNOWN,2,158,20,0,https://docs.inecelectionresults.net/elections...
3,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-004,IBEKU HALL,153,646,True,-1,False,True,False,False,UNKNOWN,4,112,22,0,https://docs.inecelectionresults.net/elections...
4,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-006,EZIALA HALL,209,750,True,-1,False,False,False,False,UNKNOWN,10,176,22,0,https://docs.inecelectionresults.net/elections...


In [ ]:
imo_crosscheck["Location"] = imo_crosscheck["PU-Name"] + ", " + imo_crosscheck["Ward"] + ", " + imo_crosscheck["LGA"]  + ", " + imo_crosscheck["State"]   + ", " + "Nigeria"


In [ ]:
lagos_crosscheck["Location"] = lagos_crosscheck["PU-Name"] + ", " + lagos_crosscheck["Ward"] + ", " + lagos_crosscheck["LGA"]  + ", " + lagos_crosscheck["State"]   + ", " + "Nigeria"


In [ ]:
def service_geocode(g_locator, address):
    location = g_locator.geocode(address)
    if location!=None:
      return location.latitude, location.longitude
    else:
      return np.NaN

In [ ]:
imo_crosscheck.shape

(2812, 20)

In [ ]:
imo_crosscheck["Lat_Long"] = imo_crosscheck["Location"].apply(lambda x: service_geocode(geolocator_arcgis, x))

In [ ]:
imo_crosscheck["Lat_Long"].tail()

In [ ]:
imo_crosscheck["Latitude"] = imo_crosscheck["Location"].apply(lambda x: geolocator_arcgis.geocode(x).latitude)
imo_crosscheck["Longitude"] = imo_crosscheck["Location"].apply(lambda x: geolocator_arcgis.geocode(x).longitude)



In [ ]:
# save imo_crosscheck yo csv

imo_crosscheck.to_csv('imo_crosscheck_with_coordinates.csv', index=False)


In [ ]:
# save imo_crosscheck yo csv

imo_crosscheck.to_csv('imo_crosscheck_with_coordinates.csv', index=False)


In [ ]:
# Reading file with coordinates

imo_crosscheck_coordinates = pd.read_csv("/content/drive/MyDrive/HNG INTERNSHIP/HNG 12/Stage 8/imo_crosscheck_with_coordinates.csv")

In [ ]:
##Checking missing data

imo_crosscheck_coordinates.isnull().sum()

In [ ]:
# drop columns

# Drop columns you don't need
columns_to_drop = ['Results_Found', 'Transcription_Count',
       'Result_Sheet_Stamped', 'Result_Sheet_Corrected',
       'Result_Sheet_Invalid', 'Result_Sheet_Unclear', 'Result_Sheet_Unsigned','Results_File']

imo_crosscheck_coordinates = imo_crosscheck_coordinates.drop(columns=columns_to_drop)



In [ ]:
imo_crosscheck_coordinates.columns

Index(['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Accredited_Voters',
       'Registered_Voters', 'APC', 'LP', 'PDP', 'NNPP', 'Location', 'Lat_Long',
       'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
data = imo_crosscheck_coordinates.copy()

## Advanced Neighbor Identification

### Geospatial Clustering (DBSCAN or HDBSCAN)

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from geopy.distance import geodesic

# Calculate distance matrix between polling units using latitude and longitude
def calculate_distance_matrix(df):
    coords = df[['Latitude', 'Longitude']].values
    distance_matrix = np.array([[geodesic(coord1, coord2).meters for coord1 in coords] for coord2 in coords])
    return distance_matrix

# DBSCAN Clustering
def perform_dbscan(distance_matrix, eps=1000, min_samples=5):  # eps=1000 for 1km proximity
    db = DBSCAN(metric="precomputed", eps=eps, min_samples=min_samples)
    clusters = db.fit_predict(distance_matrix)
    return clusters


In [ ]:

# Create distance matrix and perform DBSCAN clustering
distance_matrix = calculate_distance_matrix(data)
data['cluster'] = perform_dbscan(distance_matrix, eps=1000, min_samples=3)

# Inspect clusters
data.head()


,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,APC,LP,PDP,NNPP,Location,Lat_Long,Latitude,Longitude,cluster
0,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-001,ALADINMA UMUELEM HALL,175,905,9,40,100,3,"ALADINMA UMUELEM HALL, ENYIOGUGU, ABOH MBAISE,...","(5.420535978049, 7.240631284034)",5.420536,7.240631,0
1,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-002,OKWUAKU HALL,243,852,3,230,9,0,"OKWUAKU HALL, ENYIOGUGU, ABOH MBAISE, IMO, Nig...","(5.46475, 7.19576)",5.464750,7.195760,1
2,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-003,UMUNKWO VILLAGE HALL,183,704,2,158,20,0,"UMUNKWO VILLAGE HALL, ENYIOGUGU, ABOH MBAISE, ...","(5.420535978049, 7.240631284034)",5.420536,7.240631,0
3,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-004,IBEKU HALL,153,646,4,112,22,0,"IBEKU HALL, ENYIOGUGU, ABOH MBAISE, IMO, Nigeria","(5.46162, 7.25969)",5.461620,7.259690,2
4,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-006,EZIALA HALL,209,750,10,176,22,0,"EZIALA HALL, ENYIOGUGU, ABOH MBAISE, IMO, Nigeria","(5.46475, 7.19576)",5.464750,7.195760,1


In [ ]:
#data.to_csv('data_with_clusters.csv', index=False)  #Already downloaded and saved

In [ ]:
data.columns

Index(['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Accredited_Voters',
       'Registered_Voters', 'APC', 'LP', 'PDP', 'NNPP', 'Location', 'Lat_Long',
       'Latitude', 'Longitude', 'cluster'],
      dtype='object')

In [ ]:
import folium
import numpy as np
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster

# Generate distinct colors for clusters
def get_color(cluster_id):
    np.random.seed(cluster_id)
    return "#{:06x}".format(np.random.randint(0, 0xFFFFFF))

# Create a folium map centered at the average latitude and longitude
m = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=10)

# Add markers for each polling unit with cluster information
for _, row in data.iterrows():
    cluster_color = get_color(row['cluster']) if row['cluster'] != -1 else "#FF0000"  # Red for noise points
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=cluster_color,
        fill=True,
        fill_color=cluster_color,
        fill_opacity=0.6,
        popup=f"Polling Unit: {row['PU-Name']}<br>Cluster: {row['cluster']}"
    ).add_to(m)

# Save map as HTML
m.save("polling_units_clusters.html")


In [ ]:
"""
# Display Map in Streamlit (if applicable)
!pip install streamlit
import streamlit as st
st.write("### Polling Unit Clusters (DBSCAN Results)")
st.components.v1.html(m._repr_html_(), width=800, height=600)
"""

###  Sensitivity Analysis on Neighborhood Radii

In [ ]:
for eps_value in [500, 1000, 2000]:
    clusters = perform_dbscan(distance_matrix, eps=eps_value, min_samples=3)
    print(f'Clusters with eps={eps_value}: {len(set(clusters))}')


## Sophisticated Outlier Score Calculation

### Local Moran’s I for Spatial Autocorrelation

In [ ]:
import pysal as ps

# Prepare the spatial weights matrix (queen contiguity for neighbors)
w = ps.lib.weights.Queen.from_dataframe(data)

# Calculate Local Moran's I
mi = ps.explore.esda.Moran_Local(data['vote_percentage'], w)
data['local_moran_i'] = mi.Is

# Inspect Moran's I values
data[['polling_unit', 'local_moran_i']].head()


### Getis-Ord Gi for Hot Spot Analysis

In [ ]:
from pysal.model import spreg

# Run Getis-Ord Gi* test
g = ps.explore.esda.GetisOrd_Gi(data['vote_percentage'], w)
data['gi_star'] = g.Zs

# Inspect results
data[['polling_unit', 'gi_star']].head()


### Isolation Forest for Anomaly Detection

In [ ]:
from sklearn.ensemble import IsolationForest

# Initialize Isolation Forest
iso_forest = IsolationForest(contamination=0.05)

# Fit the model to the vote percentages or any other numerical feature
data['outlier_score'] = iso_forest.fit_predict(data[['vote_percentage']])

# Convert outlier labels to 1 (outlier) or 0 (inlier)
data['outlier_score'] = data['outlier_score'].apply(lambda x: 1 if x == -1 else 0)

# Inspect the outlier labels
data[['polling_unit', 'outlier_score']].head()


## Temporal and Demographic Comparative Analysis

In [ ]:
# Assuming 'historical_data.csv' has similar structure but for past elections
historical_data = pd.read_csv('historical_data.csv')

# Compare voting percentage over time
historical_comparison = data.merge(historical_data, on='polling_unit', suffixes=('_current', '_historical'))

# Compute voting percentage change
historical_comparison['vote_percentage_change'] = historical_comparison['vote_percentage_current'] - historical_comparison['vote_percentage_historical']

# Inspect comparison results
historical_comparison[['polling_unit', 'vote_percentage_change']].head()


For demographic analysis, you can merge with socio-economic data (e.g., income, education level) if available.

In [ ]:
# Assuming 'demographic_data.csv' contains socio-economic data by polling unit
demographic_data = pd.read_csv('demographic_data.csv')

# Merge demographic data
data = data.merge(demographic_data, on='polling_unit')

# Investigate correlation between demographic factors and voting percentages
correlations = data.corr()
print(correlations)


## Interactive Visualization and Reporting

###  Streamlit Dashboard

In [ ]:
import streamlit as st
import folium
from folium.plugins import MarkerCluster

# Create map centered on average location
m = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=10)

# Add clustering to the map
marker_cluster = MarkerCluster().add_to(m)

# Add polling unit markers
for _, row in data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Polling Unit: {row['polling_unit']}<br>Outlier Score: {row['outlier_score']}",
    ).add_to(marker_cluster)

# Display map
st.write(m)


### Visualizing Outliers and Clusters

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualizing outlier scores
sns.boxplot(x='outlier_score', y='vote_percentage', data=data)
plt.title('Outlier Detection by Vote Percentage')
plt.show()


## Reporting and Deliverables

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Create PDF report
c = canvas.Canvas("Election_Integrity_Report.pdf", pagesize=letter)

# Write content
c.drawString(100, 750, "Election Integrity Report")
c.drawString(100, 730, f"Top 5 Outliers:")
c.drawString(100, 710, "Polling Unit 1: Reason for anomaly...")
c.save()
